The following code is to read the data from a fake news dataset

In [1]:
import pandas as pd
data=pd.read_csv('train.csv')
x=data['title']
x.head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

Here, after seperating out the target column which identifies the articles in the dataset as Fake or Not Fake. It is named as the label variable and attributed to y whereas the headline is attributed to x.

In [2]:
y=data['label']
print(y.shape)
y.head()

(20800,)


0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

Modify this code Raksha...make this to output using only one line. ..well here it's just counting the number of 1s and 0s in the dataset. It would perform bettr only if the values corresponding to the two are as close as possible.

In [3]:
ones=0
zeros=0
for i in y:
    if i==1:
        ones+=1
    else:
        zeros+=1
print("ones :",ones)
print("zeros :",zeros)

ones : 10413
zeros : 10387


Here all the news headlines are bought in a list format to help in further operations.

In [4]:
sentences=[]
for i in x:
    sentences.append(str(i))
print(len(sentences))
sentences[:5]

20800


['House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It',
 'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart',
 'Why the Truth Might Get You Fired',
 '15 Civilians Killed In Single US Airstrike Have Been Identified',
 'Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery']

Here the Dataset for our purpose is prepared and split into training and testing.

In [5]:
training_size=20000
train_s = sentences[0:training_size]
test_s = sentences[training_size:]
train_labels = y[0:training_size]
test_labels = y[training_size:]

Here is where the interesting stuff happens, where we are padding to make all the lengths of the sentences equal and tokenize the text where we seperate the words out and also assign tokens aka numbers to each word available in the vocabulary.

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard

tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")
tokenizer.fit_on_texts(train_s)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_s)

train_padded = pad_sequences(train_sequences,maxlen=100,padding='post')

test_sequences = tokenizer.texts_to_sequences(test_s)

test_padded = pad_sequences(test_sequences,maxlen=100,padding='post')

In [7]:
train_padded.shape

(20000, 100)

In [16]:
import tensorflow as tf
tensorboard=TensorBoard(log_dir='logs/',profile_batch=10000000)
vocab_size=100000
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,16,input_length=100),
    #tf.keras.layers.LSTM(20),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
num_epochs = 5
model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(test_padded, test_labels),callbacks=[tensorboard])

Train on 20000 samples, validate on 800 samples
Epoch 1/5
20000/20000 [==============================] - 34s 2ms/sample - loss: 0.5835 - accuracy: 0.7297 - val_loss: 0.3689 - val_accuracy: 0.8675
Epoch 2/5
20000/20000 [==============================] - 32s 2ms/sample - loss: 0.2894 - accuracy: 0.8922 - val_loss: 0.2489 - val_accuracy: 0.8975
Epoch 3/5
20000/20000 [==============================] - 32s 2ms/sample - loss: 0.2309 - accuracy: 0.9063 - val_loss: 0.2394 - val_accuracy: 0.9038
Epoch 4/5
20000/20000 [==============================] - 29s 1ms/sample - loss: 0.2165 - accuracy: 0.9112 - val_loss: 0.2279 - val_accuracy: 0.9100
Epoch 5/5
20000/20000 [==============================] - 30s 1ms/sample - loss: 0.2121 - accuracy: 0.9118 - val_loss: 0.2379 - val_accuracy: 0.9150


Saving the neural netowrk model. when u o this a file named raxxy.h5 and raxxy.json would be created in  the directory where you are running this code...don't be scared

In [33]:
model_json=model.to_json()
with open("raxxy.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save("raxxy.h5")
print("Saved model to disk")

Saved model to disk


Getting prediction

In [30]:
new_news = ["granny starting to fear spiders in the garden might be real"]
sequences = tokenizer.texts_to_sequences(new_news)
padded = pad_sequences(sequences, maxlen=100, padding='post')
print("fake news" if model.predict(padded) >= 0.5 else "true shit")

fake news


retrieving the model and asking for the prediction 

In [34]:
from keras.models import load_model
model=load_model("raxxy.h5")
prediction=model.predict(padded)
print(prediction)
#do the above same thing in case you want lazy Raksha

[[0.71901804]]
